In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
from sithcon import SITHCon_Layer, _SITHCon_Core, iSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os 
from os.path import join
import glob
ttype=torch.cuda.FloatTensor
import numpy as np
import pandas as pd
import pickle
from math import factorial
import random
from itertools import combinations_with_replacement as comb

In [ ]:
MORSE_CODE_DICT = {'1':'.----', '2':'..---', '3':'...--', 
                    '4':'....-', '5':'.....', '6':'-....', 
                    '7':'--...', '8':'---..', '9':'----.', 
                    '0':'-----', } 

# SHORTER
MORSE_CODE_DICT = {'1':'.-', '2':'-...', 
                    '3':'-.-.', '4':'-..', '5':'.', 
                    '6':'..-.', '7':'--.', '8':'....', 
                    '9':'..', '0':'.---',}

In [ ]:

print()
morse_code_numpy = {key:np.array([int(x) for x in MORSE_CODE_DICT[key].replace('.', '10').replace('-', '1110')] + [0, 0])
                    for key in MORSE_CODE_DICT.keys()}
for k in morse_code_numpy.keys():
    print(morse_code_numpy[k], k)
subset = list(morse_code_numpy.keys())


In [ ]:
def gen_item(samples=10, scale=5, target_scale=.1):
    # We can tests 1s the same length as the items added together or 1 at the end only.
    keys = morse_code_numpy.keys()
    dim1 = []
    dim2 = []
    half = int(samples / 2)
    added_indexes = [np.random.randint(half),  
                     np.random.randint(half, samples)]
    
    answer = 0
    for s in range(samples):
        # Grab Random Morse Code Letter
        k = random.sample(keys, 1)[0]
        mcl = morse_code_numpy[k]
        Mmcl = mcl.repeat(scale)
        dim1.append(Mmcl[:, np.newaxis])
        if s in added_indexes:
            
            # dim2.append(np.ones(Mmcl.shape[0])[:, np.newaxis])
            temp = np.zeros(Mmcl.shape[0])[:, np.newaxis]
            temp[-scale:] = 1.0
            #temp[-1] = 1.0 # TRY THIS AT SOME POINT
            dim2.append(temp)
            answer += int(k)
        else:
            dim2.append(np.zeros(Mmcl.shape[0])[:, np.newaxis])
    inp = np.concatenate([np.concatenate(dim1, axis=0),
                          np.concatenate(dim2, axis=0)], axis=1)
    
    target = np.array([answer])
    return inp, target*target_scale
inp, tar = gen_item(5, 5, .1)
print(inp.shape)
plt.figure(figsize=(10,5))
plt.imshow(inp.T,aspect='auto', interpolation='none')
plt.title("Answer: {:.2f}".format(tar[0]))
plt.yticks([])
plt.savefig(join('figs', 'adding_morse_example'), dpi=100)

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

# 8Layers

In [ ]:
def gen_model(p):
    model = TCN(2, 1, [25, 25, 25, 25,25,25,25,25], kernel_size=45, dropout=0.0).cuda()
    return model

def test_model(model, X, Y):
    model.eval()
    evald = []
    evaldDict = {'test_perf':[],
                 'rate':[],
                 'tau_max':[],
                 'ntau':[],
                 'k':[]}
    
    # BIG NOTE
    # BIG NOTE
    # BIG NOTE
    # Generate the test items once, use in all models at all scales. 
    model.eval()
    evald = []
    evaldDict = {'test_perf':[],
                 'rate':[]}
    for nr in range(1,20):
        losses = []
        for iv, tar in items:

            iv = ttype(iv).unsqueeze(0).unsqueeze(0).transpose(-1,-2).unsqueeze(-1)
            iv = iv.repeat(1,1,1,1,nr)
            iv = iv.reshape(1,1,2,-1)
            tv = torch.FloatTensor(tar).to(device)
            out = model(iv)

            loss = loss_func(out[:, -1, :],
                                 tv)
            losses.append(loss.detach().cpu().numpy())
        print(nr, np.mean(losses))
        evaldDict['test_perf'].append(np.mean(losses))
        evaldDict['rate'].append(nr)
        evaldDict['tau_max'].append(model.sithcon_layers[0].sithcon.sith.tau_max)
        evaldDict['ntau'].append(model.sithcon_layers[0].sithcon.sith.ntau)
        evaldDict['k'].append(model.sithcon_layers[0].sithcon.sith.k)
    return evaldDict

def save_outcome(outcome, filename):
    dat = pd.DataFrame(outcome)
    dat.to_csv(join('perf',filename))

In [ ]:
params = [
           [None],]

In [ ]:
model = gen_model(params[0])


tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)

In [ ]:
Trainscale = 5
epochs = 30
trials_per_epoch = 1000
batch_size = 32
for r in range(5):
    model = gen_model(params[0])
    loss_func = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())


    device='cuda'
    progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')

    for epoch_idx in progress_bar:
        perfs = []
        losses = []
        model.train()
        for batch_idx in range(trials_per_epoch):
            optimizer.zero_grad()
            loss = 0
            for i in range(batch_size):
                iv, tar = gen_item(10, Trainscale, .1)
                iv = ttype(iv).unsqueeze(0).transpose(-1,-2)
                tv = ttype(tar)
                out = model(iv)
                loss += loss_func(out,
                                  tv)
            loss = loss / batch_size
            loss.backward()
            optimizer.step()

            perfs.append(0)
            #perfs = perfs[int(-loss_buffer_size/batch_size):]
            losses.append(loss.detach().cpu().numpy())
            #losses = losses[int(-loss_buffer_size/batch_size):]


            s = "{}:{:2} Loss: {:.4f}, Perf: {:.4f}"
            format_list = [epoch_idx, batch_idx, np.mean(losses), 
                           np.sum(perfs)/((len(perfs)))]
            s = s.format(*format_list)
            progress_bar.set_description(s)
    torch.save(model.state_dict(), join('perf', 'TCN_morseadding_run_{}.pt'.format(r)))

In [ ]:
items = np.load('generated_adding_morse.npy', allow_pickle=True)
loss_func = nn.MSELoss()


In [ ]:
loss_func = nn.MSELoss()


In [ ]:
losses = []
device='cuda'
for it, tar in items:
    ans = torch.FloatTensor([.9]).to(device)
    tv = torch.FloatTensor(tar).to(device)
    loss = loss_func(ans, tv)
    losses.append(loss.detach().cpu().numpy())
plt.hist(losses)
print(np.mean(losses))

In [ ]:
runs = 5
device='cuda'
for r in range(runs):
    model.load_state_dict(torch.load(join('perf','TCN_morseadding_run_{}.pt'.format(r))))
    model.eval()
    evald = []
    evaldDict = {'test_perf':[],
                 'rate':[]}
    for nr in [1,3,4,5,6,12,25,50]:
    #for nr in range(1,40,):
        perfs = []
        losses = []
        for iv, tar in items:
            iv = ttype(iv).unsqueeze(0).transpose(-1,-2).unsqueeze(-1)
            iv = iv.repeat(1,1,1,nr)
            iv = iv.reshape(1,2,-1)
            tv = torch.FloatTensor(tar).to(device)
            out = model(iv)

            loss = loss_func(out,
                                 tv)
            losses.append(loss.detach().cpu().numpy())
        print(nr, np.mean(losses))
        evaldDict['test_perf'].append(np.mean(losses))
        evaldDict['rate'].append(nr)
        evald.append([nr, np.mean(losses)])
    scale_perfs = pd.DataFrame(evaldDict)
    scale_perfs.to_pickle(join("perf", "TCN_morseadding_test_{}.dill".format(r)))

In [ ]:
TCN_perfs = None
for r in range(5):
    if TCN_perfs is None:
        TCN_perfs = pd.read_pickle(join("perf", "TCN_morseadding_test_{}.dill".format(r)))
        TCN_perfs['run'] = [r]*TCN_perfs.shape[0]
    else:
        temp =  pd.read_pickle(join("perf", "TCN_morseadding_test_{}.dill".format(r)))
        temp['run'] = [r]*temp.shape[0]
        TCN_perfs = TCN_perfs.append(temp)
TCN_perfs['adj_scale'] = TCN_perfs.rate / Trainscale
print(TCN_perfs)

In [ ]:

window_size = 20
lw=5
ms = 15
with sn.plotting_context("notebook", font_scale=2):
    fig = plt.figure(figsize=(20,10), )
    sn.lineplot(data=TCN_perfs, x='adj_scale', y='test_perf', 
                markers=True, linewidth=lw, ms=ms,)
    plt.ylim(0, 1.0)